<a href="https://colab.research.google.com/github/Zahra-Ferdous/Thesis_Human_Action_Recognition_from_Depth_Video/blob/main/depth_video_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os
import time

from numba.core.cgutils import if_likely
#from os.path import exists
import os.path 

import cv2
import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
from scipy.io import loadmat


import imageio
from skimage import feature

In [3]:
'''
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='22') or (file[0:2]=='cx' and file[4:6]=='22')):
        d = sio.loadmat(path+'/'+file)['depth']
        #print(cnt)
        print(file)
      
        print(d.shape)
'''

"\npath = '/content/drive/MyDrive/depth_mat'\n\ncnt = 0\n#max_frame = -1\nstart = time.time()\n\nfor file in os.listdir(path):\n    if file.endswith('.mat'):\n      cnt+=1\n      # print(cnt)\n      # print(file)\n\n      if(cnt == 13):\n        continue\n      \n      if((file[0:2]!='cx' and file[5:7]=='22') or (file[0:2]=='cx' and file[4:6]=='22')):\n        d = sio.loadmat(path+'/'+file)['depth']\n        #print(cnt)\n        print(file)\n      \n        print(d.shape)\n"

In [4]:
'''
path = '/content/drive/MyDrive/depth_mat/zyh_a22_t3.mat'
#dat = sio.loadmat(path)['depth']
d = sio.loadmat(path)['depth']
frame = d.shape[0]
print(frame)
for k in range(frame):
  if(k%1==0):
    plt.imshow(d[k], interpolation='nearest')
    plt.title(k)
    plt.show()

          # plt.imshow(d[k], 
#dat.keys()
'''

"\npath = '/content/drive/MyDrive/depth_mat/zyh_a22_t3.mat'\n#dat = sio.loadmat(path)['depth']\nd = sio.loadmat(path)['depth']\nframe = d.shape[0]\nprint(frame)\nfor k in range(frame):\n  if(k%1==0):\n    plt.imshow(d[k], interpolation='nearest')\n    plt.title(k)\n    plt.show()\n\n          # plt.imshow(d[k], \n#dat.keys()\n"

In [5]:
'''
p = '/content/drive/MyDrive/depth_mat/myj_a1_t2.mat'
d = sio.loadmat(p)['depth']
frame = d.shape[0]
print(frame)
for k in range(30,130):
        if k%2==0:
          
          im = np.array(d[k][100:370, 150:400])
          #print(im)
          im = cv2.resize(im, dsize=(224,224))
          im = np.array([im])
          #im = np.array()
          if(k==30):
            im1 = im
          else:
            im1 = np.concatenate((im1, im), axis=0)

          # plt.imshow(d[k], interpolation='nearest')
          # plt.title(k)
          # plt.show()
print(im1.shape)
ser_data = {'data':im1,'label':1} 
ser = pd.Series(data = ser_data, index = ['data','label'])
l_ser.append(ser)
'''

"\np = '/content/drive/MyDrive/depth_mat/myj_a1_t2.mat'\nd = sio.loadmat(p)['depth']\nframe = d.shape[0]\nprint(frame)\nfor k in range(30,130):\n        if k%2==0:\n          \n          im = np.array(d[k][100:370, 150:400])\n          #print(im)\n          im = cv2.resize(im, dsize=(224,224))\n          im = np.array([im])\n          #im = np.array()\n          if(k==30):\n            im1 = im\n          else:\n            im1 = np.concatenate((im1, im), axis=0)\n\n          # plt.imshow(d[k], interpolation='nearest')\n          # plt.title(k)\n          # plt.show()\nprint(im1.shape)\nser_data = {'data':im1,'label':1} \nser = pd.Series(data = ser_data, index = ['data','label'])\nl_ser.append(ser)\n"

In [6]:
#ser_data

In [7]:
# if(dat.shape[0]>100):
#   extra = dat.shape[0] - 100
l_ser = []

## a1 = 54 (right hand wave)

In [8]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13 or cnt == 1):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='1_') or (file[0:2]=='cx' and file[4:6]=='1_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        #print(d.shape)
        frame = d.shape[0]
        if(frame<130):
          for k in range(0,100):
            if k%2==0:
          
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(30,130):
            if k%2==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==30):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        #print(im1.shape)
        im1 = im1/im1.max()
        
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        # print(im1.shape)
        ser_data = {'data':im1,'label':1} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


##a2 = 53 (left hand wave)

In [9]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='2_') or (file[0:2]=='cx' and file[4:6]=='2_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if(frame<130):
          for k in range(16,116):
            if k%2==0:
          
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==16):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(30,130):
            if k%2==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==30):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':2} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a3 = 53 (Right horizontal wave)

In [10]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13 ):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='3_') or (file[0:2]=='cx' and file[4:6]=='3_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if(frame<130):
          for k in range(14,114):
            if k%2==0:
          
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==14):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(30,130):
            if k%2==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==30):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':3} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a4 = 53 (left horizontal wave)

In [11]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='4_') or (file[0:2]=='cx' and file[4:6]=='4_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if(frame<130):
          for k in range(14,114):
            if k%2==0:
          
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==14):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(30,130):
            if k%2==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==30):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':4} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a5 = 53 ( Hammer with right hand )

In [12]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
cxxnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13 ):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='5_') or (file[0:2]=='cx' and file[4:6]=='5_')):
        d = sio.loadmat(path+'/'+file)['depth']
        cxxnt += 1
        # if(cxxnt <= 23 ):
        #   continue
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(0,50):
          if k%1==0:
            im = np.array(d[k][100:370, 150:400])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==0):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':5} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a6 = Grasp with right hand

In [13]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13 ):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='6_') or (file[0:2]=='cx' and file[4:6]=='6_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(8,58):
          if k%1==0:
            im = np.array(d[k][100:370, 150:400])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==8):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':6} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a7 = Draw x with right hand

In [14]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='7_') or (file[0:2]=='cx' and file[4:6]=='7_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        print(d.shape)
        frame = d.shape[0]
        if(frame<100):
          for k in range(10,60):
            if k%1==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==10):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,100):
            if k%2==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':7} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

1
myj_a18_t2.mat
2
myj_a18_t4.mat
3
myj_a18_t1.mat
4
myj_a18_t3.mat
5
myj_a18_t5.mat
6
myj_a18_t6.mat
7
myj_a18_t9.mat
8
myj_a19_t1.mat
9
myj_a18_t8.mat
10
myj_a18_t7.mat
11
myj_a19_t10.mat
12
myj_a19_t2.mat
13
myj_a19_t7.mat
14
myj_a19_t4.mat
15
myj_a19_t3.mat
16
myj_a19_t5.mat
17
myj_a19_t6.mat
18
myj_a19_t9.mat
19
myj_a19_t8.mat
20
myj_a1_t2.mat
21
myj_a1_t10.mat
22
myj_a1_t1.mat
23
myj_a1_t4.mat
24
myj_a1_t5.mat
25
myj_a1_t3.mat
26
myj_a1_t6.mat
27
myj_a1_t8.mat
28
myj_a1_t7.mat
29
myj_a1_t9.mat
30
myj_a20_t1.mat
31
myj_a20_t2.mat
32
myj_a20_t10.mat
33
myj_a20_t5.mat
34
myj_a20_t3.mat
35
myj_a20_t4.mat
36
myj_a20_t7.mat
37
myj_a20_t6.mat
38
myj_a20_t9.mat
39
myj_a20_t8.mat
40
myj_a21_t2.mat
41
myj_a21_t1.mat
42
myj_a21_t10.mat
43
myj_a21_t5.mat
44
myj_a21_t4.mat
45
myj_a21_t3.mat
46
myj_a21_t6.mat
47
myj_a21_t8.mat
48
myj_a21_t7.mat
49
myj_a22_t1.mat
50
myj_a22_t10.mat
51
myj_a21_t9.mat
52
myj_a22_t3.mat
53
myj_a22_t2.mat
54
myj_a22_t4.mat
55
myj_a22_t7.mat
56
myj_a22_t5.mat
57
myj

## a8 = Draw x with left hand

In [15]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='8_') or (file[0:2]=='cx' and file[4:6]=='8_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if(frame<100):
          for k in range(2,23):
            if k%2==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==2):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(23,52):
            if k%1==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              
              im1 = np.concatenate((im1, im), axis=0)
          for k in range(52,71):
            if k%2==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,100):
            if k%2==0:
              im = np.array(d[k][100:370, 10:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':8} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a9 = Draw circle with right hand 

In [16]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='9_') or (file[0:2]=='cx' and file[4:6]=='9_')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if(frame<60):
          for k in range(7,57):
            if k%1==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==7):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        elif frame<100:
          for k in range(12,62):
            if k%1==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==12):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,100):
            if k%2==0:
              im = np.array(d[k][100:370, 150:400])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':9} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a10 = Draw circle with left hand 

In [17]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='10') or (file[0:2]=='cx' and file[4:6]=='10')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]

        if(frame<50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          im = np.zeros((224,224),np.int32)
          im = np.array([im])
          im1 = np.concatenate((im1, im), axis=0)

        elif(frame<100):
          for k in range(7,57):
            if k%1==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==7):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)

        else:
          for k in range(0,100):
            if k%2==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':10} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a11 = Right foot kick forward

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='11') or (file[0:2]=='cx' and file[4:6]=='11')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:400, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,50):
            if k%1==0:
              im = np.array(d[k][100:400, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':11} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

(50, 224, 224, 1)
(50, 224, 224, 1)
(50, 224, 224, 1)


## a12 = left foot kick forward

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='12') or (file[0:2]=='cx' and file[4:6]=='12')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:400, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)

        else:
          for k in range(0,50):
            if k%1==0:
              im = np.array(d[k][100:400, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':12} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a13 = Right foot kick side

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='13') or (file[0:2]=='cx' and file[4:6]=='13')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:370, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,50):
            if k%1==0:
              im = np.array(d[k][100:370, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1) 
        print(im1.shape)
        ser_data = {'data':im1,'label':13} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a14 = left foot kick side

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='14') or (file[0:2]=='cx' and file[4:6]=='14')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,50):
            if k%1==0:
              im = np.array(d[k][100:370, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':14} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a15 = clap

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='15') or (file[0:2]=='cx' and file[4:6]=='15')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:400, 120:370])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(10,60):
            if k%1==0:
              im = np.array(d[k][100:400, 100:350])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==10):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':15} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a16 = Bend down

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='16') or (file[0:2]=='cx' and file[4:6]=='16')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]

        if(frame<=50):
          for k in range(0,frame):
            if k%1==0:
              im = np.array(d[k][100:400, 120:370])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,50):
            if k%1==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)

        elif(frame<100):
          for k in range(0,50):
            if k%1==0:
              im = np.array(d[k][100:370, 120:370])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)

        else:
          for k in range(0,100):
            if k%2==0:
              im = np.array(d[k][100:370, 120:370])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':16} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a17 = Wave up and down

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='17') or (file[0:2]=='cx' and file[4:6]=='17')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(12,112):
          if k%2==0:
        
            im = np.array(d[k][70:370, 100:350])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==12):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':17} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a18 = Sur Place

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='18') or (file[0:2]=='cx' and file[4:6]=='18')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        if frame >= 100:
          for k in range(0,100):
            if k%2==0:
          
              im = np.array(d[k][100:370, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
        else:
          for k in range(0,frame):
            if k%2==0:
              im = np.array(d[k][100:370, 140:390])
              #print(im)
              im = cv2.resize(im, dsize=(224,224))
              im = np.array([im])
              #im = np.array()
              if(k==0):
                im1 = im
              else:
                im1 = np.concatenate((im1, im), axis=0)
          for k in range(frame,100):
            if k%2==0:
              im = np.zeros((224,224),np.int32)
              im = np.array([im])
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':18} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a19 = Left body turning movement

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='19') or (file[0:2]=='cx' and file[4:6]=='19')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(12,112):
          if k%2==0:
        
            im = np.array(d[k][100:370, 100:350])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==12):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':19} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## a20 = Right body turning movement

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='20') or (file[0:2]=='cx' and file[4:6]=='20')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(6,106):
          if k%2==0:
        
            im = np.array(d[k][100:370, 140:390])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==6):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':20} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

##a21 = Left lateral movement

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='21') or (file[0:2]=='cx' and file[4:6]=='21')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(12,112):
          if k%2==0:
        
            im = np.array(d[k][70:370, 100:390])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==12):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':21} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

##a22 = Right lateral movement

In [ ]:
path = '/content/drive/MyDrive/depth_mat'

cnt = 0
#max_frame = -1
start = time.time()

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      # print(cnt)
      # print(file)

      if(cnt == 13):
        continue
      
      if((file[0:2]!='cx' and file[5:7]=='22') or (file[0:2]=='cx' and file[4:6]=='22')):
        d = sio.loadmat(path+'/'+file)['depth']
      
        # print(d.shape)
        frame = d.shape[0]
        
        for k in range(12,112):
          if k%2==0:
        
            im = np.array(d[k][70:370, 100:390])
            #print(im)
            im = cv2.resize(im, dsize=(224,224))
            im = np.array([im])
            #im = np.array()
            if(k==12):
              im1 = im
            else:
              im1 = np.concatenate((im1, im), axis=0)
        # im1 = im1/im1.max()
        # im1 = im1.reshape(im1.shape[0], 224,224, 1)
        print(im1.shape)
        ser_data = {'data':im1,'label':22} 
        ser = pd.Series(data = ser_data, index = ['data','label'])
        l_ser.append(ser)

## series

In [ ]:
len(l_ser)

In [ ]:
cols = ['data','label']
df = pd.DataFrame(l_ser, columns=cols)
print(df.shape)
df.head(10)

(1165, 2)


,data,label
0,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
1,"[[[116, 117, 116, 116, 116, 116, 117, 117, 117...",1
2,"[[[116, 116, 116, 116, 117, 117, 116, 116, 116...",1
3,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
4,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
5,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
6,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
7,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
8,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1
9,"[[[116, 116, 116, 116, 116, 116, 116, 116, 116...",1


In [ ]:
daf = sio.loadmat('/content/drive/MyDrive/Colab Notebooks/predictions/dmm_front_train_test_224x224.mat')

#daf = sio.loadmat('/content/drive/MyDrive/Colab Notebooks/predictions/dmm_front_train_test_norm_224x224.mat')
print(daf.keys())

dict_keys(['__header__', '__version__', '__globals__', 'train_x', 'test_x', 'train_y', 'test_y'])


In [ ]:
sio.savemat('/content/drive/MyDrive/Colab Notebooks/depth_video_50x224x224.mat',{name: col.values for name, col in df.items()})